## Order supplies dataset load
### Agenda
* Dataset description
* Cleaning
* Profit outlier detection and handling
* Country names normalization
* Final corrections


### Dataset description
The DataCo Supply Chain Dataset contains detailed transactional data related to a global retail supply chain.
Each record represents an order line item, combining information about orders, customers, products, shipping, and financial performance.

**The dataset includes**:
* Order details such as order dates, status, region, and market
* Customer information including customer segment and location
* Product attributes such as product name, category, price, and quantity
* Shipping and logistics data, including shipping mode, scheduled vs. actual delivery time, and delivery status
* Financial metrics, including sales, profit, discounts, and profit ratios
* Geographic information, allowing country-level and regional analysis

The data spans multiple years (2015-2018) and multiple countries, making it suitable for analyzing:
* Sales and profit performance
* Customer and product behavior
* Shipping efficiency and delivery delays
* Geographic and market-level trends

### Cleaning
This part includes basic cleaning steps like: droppping, renaming columns, deleting duplicates and creating new columns

In [1]:
import pandas as pd
import pycountry
import os
from deep_translator import GoogleTranslator
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
#Load data

raw_df = pd.read_csv("raw_data\DataCoSupplyChainDataset1.csv")
raw_df.head()

Error importing huggingface_hub.hf_file_system: No module named 'cgi'


ImportError: Unable to load filesystem from EntryPoint(name='hf', value='huggingface_hub.HfFileSystem', group='fsspec.specs')

In [13]:
raw_df.columns

Index(['Type', 'Days_for_shipping_(real)', 'Days_for_shipment_(scheduled)',
       'Benefit_per_order', 'Sales_per_customer', 'Delivery_Status',
       'Late_delivery_risk', 'Category_Id', 'Category_Name', 'Customer_City',
       'Customer_Country', 'Customer_Email', 'Customer_Fname', 'Customer_Id',
       'Customer_Lname', 'Customer_Password', 'Customer_Segment',
       'Customer_State', 'Customer_Street', 'Customer_Zipcode',
       'Department_Id', 'Department_Name', 'Latitude', 'Longitude', 'Market',
       'Order_City', 'Order_Country', 'Order_Customer_Id',
       'order_date_(DateOrders)', 'Order_Id', 'Order_Item_Cardprod_Id',
       'Order_Item_Discount', 'Order_Item_Discount_Rate', 'Order_Item_Id',
       'Order_Item_Product_Price', 'Order_Item_Profit_Ratio',
       'Order_Item_Quantity', 'Sales', 'Order_Item_Total',
       'Order_Profit_Per_Order', 'Order_Region', 'Order_State', 'Order_Status',
       'Order_Zipcode', 'Product_Card_Id', 'Product_Category_Id',
       'Product_De

In [14]:
#Delete usless columns
raw_df.drop(columns=['Benefit_per_order', 'Sales_per_customer', 'Late_delivery_risk', 'Category_Id', 'Customer_City',
                     'Customer_Email', 'Customer_Id', 'Customer_Password', 
                     'Customer_Country', 'Department_Name', 'Customer_Fname', 'Customer_Lname',
                     'Customer_State', 'Customer_Street', 'Customer_Zipcode', 'Department_Id', 'Latitude', 'Longitude',
                     'Order_City', 'Order_Customer_Id', 'Order_Item_Cardprod_Id', 'Order_Item_Discount', 'Order_Item_Discount_Rate',
                     'Order_Item_Id', 'Order_Item_Profit_Ratio', 'Order_Item_Total', 'Order_State', 'Order_Zipcode',
                     'Product_Category_Id', 'Product_Description', 'Product_Image', 'Product_Status'], inplace = True)

#Rename remaining columns
raw_df.rename(columns={'shipping_date_(DateOrders)': 'Shipping_Date', 'order_date_(DateOrders)': 'Order_Date', 
                       'Type': 'Payment_Type', 'Days_for_shipping_(real)': 'Shipment(real)', 'Days_for_shipment_(scheduled)': 'Shipment(scheduled)',
                       'Order_Item_Product_Price': 'Item_Price', 'Order_Item_Quantity': 'Item_Quantity', 
                       'Order_Profit_Per_Order': 'Order_Profit', 'Product_Card_Id': 'Product_Id'}, inplace=True)

In [ ]:
#Calculate delays
raw_df['Delay_Days'] = raw_df['Shipment(real)'] - raw_df['Shipment(scheduled)']

Q1 = raw_df['Delay_Days'].quantile(0.25)
Q3 = raw_df['Delay_Days'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR  
upper_bound = Q3 + 1.5 * IQR 

raw_df['Delay_Type'] = 'Normal'
raw_df.loc[raw_df['Delay_Days'] > upper_bound, 'Delay_Type'] = 'Late'
raw_df.loc[raw_df['Delay_Days'] < lower_bound, 'Delay_Type'] = 'Early'

In [15]:
#There are 0 null values; 0 duplicate rows

nulls = raw_df.isna().sum()
duplicates = raw_df.duplicated().sum()

if duplicates != 0:
    raw_df.drop_duplicates(inplace=True)
    
print(f'Null values:\n{nulls}\nDuplicate rows: {duplicates}')

Null values:
Payment_Type           0
Shipment(real)         0
Shipment(scheduled)    0
Delivery_Status        0
Category_Name          0
Customer_Segment       0
Market                 0
Order_Country          0
Order_Date             0
Order_Id               0
Item_Price             0
Item_Quantity          0
Sales                  0
Order_Profit           0
Order_Region           0
Order_Status           0
Product_Id             0
Product_Name           0
Product_Price          0
Shipping_Date          0
Shipping_Mode          0
dtype: int64
Duplicate rows: 8


In [16]:
#Convert date columns into datetime datatype
raw_df['Shipping_Date'] = pd.to_datetime(raw_df['Shipping_Date'], errors='coerce')
raw_df['Order_Date'] = pd.to_datetime(raw_df['Order_Date'], errors='coerce')

raw_df.dtypes

Payment_Type                   object
Shipment(real)                  int64
Shipment(scheduled)             int64
Delivery_Status                object
Category_Name                  object
Customer_Segment               object
Market                         object
Order_Country                  object
Order_Date             datetime64[ns]
Order_Id                        int64
Item_Price                      int64
Item_Quantity                   int64
Sales                         float64
Order_Profit                  float64
Order_Region                   object
Order_Status                   object
Product_Id                      int64
Product_Name                   object
Product_Price                 float64
Shipping_Date          datetime64[ns]
Shipping_Mode                  object
dtype: object

### Profit outlier detection and handling
The Order Profit column is filtered for outliers to prevent extreme values from distorting subsequent profit analyses.

In [17]:
#Detecting outliers
Q1 = raw_df['Order_Profit'].quantile(0.25)
Q3 = raw_df['Order_Profit'].quantile(0.75)
IQR = Q3 - Q1

outliers = raw_df[(raw_df['Order_Profit'] < Q1 - 1.5*IQR) | (raw_df['Order_Profit'] > Q3 + 1.5*IQR)]
percentage = len(outliers)/len(raw_df['Order_Profit'])*100

print(f'The number of outliers: {len(outliers)} ({round(percentage,2)}%)')

The number of outliers: 18942 (10.49%)


After identifying outliers in the dataset, approximately **10%** of the observations were classified as extreme values. These observations are likely to represent valid but extreme cases rather than data errors. 
To reduce their influence on further analyses, outliers will be capped using the ***1st–99th percentile boundary (winsorization)***. This approach limits the impact of extreme values without distorting the overall data structure.

In [18]:
#Winsorization
lower = raw_df['Order_Profit'].quantile(0.01) 
upper = raw_df['Order_Profit'].quantile(0.99)

raw_df['Profit_Capped'] = raw_df['Order_Profit'].clip(lower, upper)
raw_df.drop(columns='Order_Profit', inplace=True)

In [ ]:
#Reorder columns for further analysis
new_order = [#Order
    'Order_Id', 'Order_Date', 'Order_Status', 'Order_Region', 'Market', 'Order_Country',
    #Customer
    'Customer_Segment',
    #Product 
    'Product_Id', 'Product_Name', 'Product_Price', 'Category_Name',
    #Shipping
    'Shipping_Date', 'Shipping_Mode', 'Shipment(real)', 'Shipment(scheduled)', 'Delivery_Status', 
    'Payment_Type', 'Delay_Days',
    #Finances
    'Item_Price', 'Item_Quantity', 'Sales', 'Profit_Capped']
ordered_df = raw_df[[c for c in new_order if c in raw_df.columns]]
ordered_df.columns

Index(['Order_Id', 'Order_Date', 'Order_Status', 'Order_Region', 'Market',
       'Order_Country', 'Customer_Segment', 'Product_Id', 'Product_Name',
       'Product_Price', 'Category_Name', 'Shipping_Date', 'Shipping_Mode',
       'Shipment(real)', 'Shipment(scheduled)', 'Delivery_Status',
       'Payment_Type', 'Item_Price', 'Item_Quantity', 'Sales',
       'Profit_Capped'],
      dtype='object')

### Country names normalization
To enable merging the orders dataset with GDP and population data, country names must first be normalized. Since some country names are not in English, they are translated where necessary and then converted to **ISO3** country codes.

In [ ]:
#Normalize country names
ordered_df['Order_Country_Normalized'] = ordered_df['Order_Country'].str.lower().str.strip()

manual_fixes = {
    'martinica': 'france',
    'guadalupe': 'france',
    'french guiana': 'france',
    'sahara occidental': None
}

ordered_df['Order_Country_Normalized'] = ordered_df['Order_Country_Normalized'].replace(manual_fixes)

#Assign the ISO code to each country
def get_iso3(name):
    if pd.isna(name):
        return None
    try:
        return pycountry.countries.lookup(name).alpha_3
    except LookupError:
        return None

ordered_df['ISO3'] = ordered_df['Order_Country_Normalized'].apply(get_iso3)

C:\Users\user\AppData\Local\Temp\ipykernel_1532\3337880426.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ordered_df['Order_Country_Normalized'] = ordered_df['Order_Country'].str.lower().str.strip()
C:\Users\user\AppData\Local\Temp\ipykernel_1532\3337880426.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ordered_df['Order_Country_Normalized'] = ordered_df['Order_Country_Normalized'].replace(manual_fixes)
C:\Users\user\AppData\Local\Temp\ipykernel_1532\3337880426.py:22: SettingWithCopyWarning: 
A v

In [ ]:
#Mismatched countries count
mis_len = len(ordered_df[ordered_df['ISO3'].isna()]['Order_Country_Normalized'].unique())
print(f'The number of mismatched country names is {mis_len}')

86

In [23]:
ordered_df['Order_Country_Normalized'] = ordered_df['Order_Country'].str.strip().str.title()

mismatched = ordered_df[ordered_df['ISO3'].isna()]['Order_Country_Normalized'].unique()

#Fixes for territories
manual_translations = {
    'Martinica': 'France',
    'Guadalupe': 'France',
    'French Guiana': 'France',
    'Sahara Occidental': None,
    'Hong Kong': 'China',
    'Taiwan': 'Taiwan',
}

#Auto-translate
to_translate = [c for c in mismatched if c not in manual_translations and pd.notna(c)]
translated_list = GoogleTranslator(source='auto', target='en').translate_batch(to_translate)
translated_mapping = dict(zip(to_translate, translated_list))
translated_mapping.update(manual_translations)  # combine

#Map normalized and translated names
ordered_df['Order_Country_Translated'] = ordered_df['Order_Country_Normalized'].map(
    lambda x: translated_mapping.get(x, x))

#Map them to ISO3
def country_to_iso(name):
    if name is None:
        return None
    try:
        return pycountry.countries.lookup(name).alpha_3
    except LookupError:
        return None

ordered_df['ISO3_Final'] = ordered_df['Order_Country_Translated'].apply(country_to_iso)

#Check for remaining unmatched
remaining_unmatched = ordered_df[ordered_df['ISO3_Final'].isna()]['Order_Country_Translated'].unique()
print("Countries still needing manual handling:")
print(remaining_unmatched)


C:\Users\user\AppData\Local\Temp\ipykernel_1532\3070785409.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ordered_df['Order_Country_Normalized'] = ordered_df['Order_Country'].str.strip().str.title()
C:\Users\user\AppData\Local\Temp\ipykernel_1532\3070785409.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ordered_df['Order_Country_Translated'] = ordered_df['Order_Country_Normalized'].map(


Countries still needing manual handling:
['Democratic Republic of Congo' 'Ivory Coast' 'Italia' 'Espana' 'Irak'
 'Arabia Saudi' 'Filipinas' 'Myanmar (Burmania)' 'Rumania'
 'Trinity of Y Tobago' 'Russia' 'Belgica' 'Kirguistan' 'Camboya' 'Lesoto'
 'Swiss' 'Barein' 'Macedonia' 'Azerbaiyan' 'Moldavia' 'Tune' 'Yibuti'
 'Swaziland' 'Surinam' 'Belice' 'Republic Of Gambia' None]


C:\Users\user\AppData\Local\Temp\ipykernel_1532\3070785409.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ordered_df['ISO3_Final'] = ordered_df['Order_Country_Translated'].apply(country_to_iso)


In [24]:
#Manual translation
manual_translations_final = {
    'Democratic Republic of Congo': 'Congo',
    'Ivory Coast': 'Côte d\'Ivoire',
    'Italia': 'Italy',
    'Espana': 'Spain',
    'Irak': 'Iraq',
    'Arabia Saudi': 'Saudi Arabia',
    'Filipinas': 'Philippines',
    'Myanmar (Birmania)': 'Myanmar',
    'Rumania': 'Romania',
    'Trinity of Y Tobago': 'Trinidad and Tobago',
    'Russia': 'Russian Federation',
    'Belgica': 'Belgium',
    'Kirguistan': 'Kyrgyzstan',
    'Camboya': 'Cambodia',
    'Lesoto': 'Lesotho',
    'Swiss': 'Switzerland',
    'Barein': 'Bahrain',
    'Macedonia': 'North Macedonia',
    'Azerbaiyan': 'Azerbaijan',
    'Moldavia': 'Moldova',
    'Tune': 'Tunisia',
    'Yibuti': 'Djibouti',
    'Swaziland': 'Eswatini',
    'Surinam': 'Suriname',
    'Belice': 'Belize',
    'Republic Of Gambia': 'Gambia',
    None: None
}

#Final translation
ordered_df['Order_Country_Translated_Final'] = ordered_df['Order_Country_Translated'].map(
    lambda x: manual_translations_final.get(x, x)
)
ordered_df = ordered_df.dropna(subset=['Order_Country_Translated_Final'])

#Map to ISO3
ordered_df['ISO3_Final'] = ordered_df['Order_Country_Translated_Final'].apply(country_to_iso)

#Check for remaining unmatched
remaining_unmatched = ordered_df[ordered_df['ISO3_Final'].isna()]['Order_Country_Translated_Final'].unique()
print("Countries still needing manual handling:")
print(remaining_unmatched)


C:\Users\user\AppData\Local\Temp\ipykernel_1532\1106431376.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ordered_df['Order_Country_Translated_Final'] = ordered_df['Order_Country_Translated'].map(


Countries still needing manual handling:
['Myanmar (Burmania)']


### Final corrections
The final step is to reorder columns and save the datframe to a csv file.

In [25]:
#Drop and rename columns

ordered_df.drop(columns=['Order_Country', 'Order_Country_Normalized', 'ISO3', 'Order_Country_Translated', 'Order_Country_Translated_Final'], inplace=True)
ordered_df.rename(columns={'ISO3_Final': 'ISO3'}, inplace=True)

#Reorder columns for further analysis

new_order = [#Order
    'Order_Id', 'Order_Date', 'Order_Status', 'Order_Region', 'Market', 'ISO3',
    #Customer
    'Customer_Segment',
    #Product 
    'Product_Id', 'Product_Name', 'Category_Name', 'Product_Price',
    #Shipping
    'Shipping_Date', 'Shipping_Mode', 'Shipment(real)', 'Shipment(scheduled)', 'Delivery_Status', 'Payment_Type',
    #Finances
    'Item_Price', 'Item_Quantity', 'Sales', 'Profit_Capped']
ordered_df = ordered_df[[c for c in new_order if c in ordered_df.columns]]

ordered_df.columns

Index(['Order_Id', 'Order_Date', 'Order_Status', 'Order_Region', 'Market',
       'ISO3', 'Customer_Segment', 'Product_Id', 'Product_Name',
       'Category_Name', 'Product_Price', 'Shipping_Date', 'Shipping_Mode',
       'Shipment(real)', 'Shipment(scheduled)', 'Delivery_Status',
       'Payment_Type', 'Item_Price', 'Item_Quantity', 'Sales',
       'Profit_Capped'],
      dtype='object')

In [26]:
#Save to csv
csv_file = os.path.join('transformed', 'orders_cleaned.csv')
os.makedirs(os.path.dirname(csv_file), exist_ok=True)

ordered_df.to_csv(csv_file, index=False)

print(f'Data saved to {csv_file}')

Data saved to transformed\orders_cleaned.csv
